<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

# US Institutions Financial Report

In [1]:
# May need to install pandas directly in the Jupyter notebook with the following commands
#! pip3 install --user pandas
#! pip3 install --user psycopg
#!pip3 install --user matplotlib
import pandas as pd
import psycopg
import matplotlib as plt
from credentials import DBNAME, HOST, USERNAME, PASSWORD

In [2]:
# Parameters
YEAR = 2021


In [3]:
# Set the connection and grab the data from SQL

# Connect to the database using parameters from credentials.py
conn = psycopg.connect(
    dbname=DBNAME,
    host=HOST,
    user=USERNAME,
    password=PASSWORD)

# Create a cursor and use it to submit/execute a query:
cur = conn.cursor()

select_cmd = """
    SELECT * FROM institutions 
    WHERE extracted_year <= to_date(%s::text ,'YYYY-MM-DD');
"""

# Save to a dataframe
df = pd.DataFrame(cur.execute(select_cmd, (YEAR,)))
df.columns =['pkey', 'opeid', 'name', 'address', 'latitude',
            'longitude', 'fips', 'region', 'ccbasic', 'cbsa', 
            'csa', 'accreditor', 'pred_degree', 'highest_degree',
            'control', 'admission_rate', 'in_state_tuit', 'out_state_tuit',
            'prog_year_tuit', 'revenue_tuit', 'two_yr_default',
            'three_yr_default', 'avg_faculty_salary', 'sat_avg', 
            'prop_loan', 'extracted_year', 'zip', 'city', 'state']

# Close the connection
cur.close()
conn.close()

# Data Summary

# Summary of Current College Tuition Rates

# Best and Worst Performing Institutions by Loan-Repayment Rates

In [4]:
# Sort the DataFrame by loan repayment rates in ascending order
df_sorted = df.sort_values(by='three_yr_default')

# Extract the top 10 and bottom 10 institutions
best_performing = df_sorted.head(10)
worst_performing = df_sorted.tail(10)

#print(best_performing)
#print(worst_performing)

In [5]:
# Can export to a PDF if desired
# OPTIONAL: don't need to run
best_performing[['name', 'three_yr_default']].to_csv('data/best_performing_institutions.csv', index=False)
worst_performing[['name', 'three_yr_default']].to_csv('data/worst_performing_institutions.csv', index=False)

# Tuition and Loan Repayment Rates over Time

How have tuition rates and loan repayment rates changed over time for the top 10 most expensive universities in the U.S.?
We will explore this by first finding the top 10 most expensive schools.

In [6]:
# First find the top 10 most expensive schools
# Find most expensive out of:
# (in_state_tuit + out_state_tuit)/2, or the average tuition cost

df['avg_tuition'] = (df['in_state_tuit'] + df['out_state_tuit']) / 2

df_sorted = df.sort_values(by='avg_tuition', ascending=False)

top_10 = df_sorted.head(10)

Next, graph these schools based on their tuition and loan repayment rates.

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:
plt.figure(figsize=(10, 6))
plt.scatter(top_10['avg_tuition'], top_10['three_yr_default'], color='blue')
plt.title('Top 10 Most Expensive Universities - Tuition vs. Three-Year Default Rate')
plt.xlabel('Average Tuition')
plt.ylabel('Three-Year Default Rate')
plt.show()

AttributeError: module 'matplotlib' has no attribute 'figure'

# Relationship between average tuition and revenue tuition and SAT score

Are “smarter” colleges more expensive? Below, we have plots of the average tuition vs. SAT score as well as the revenue tuition vs. SAT score.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df['sat_avg'], df['avg_tuition'], color='red')
plt.title('Average Tuition vs. SAT scores')
plt.xlabel('SAT score')
plt.ylabel('Average tuition')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df['sat_avg'], df['revenue_tuit'], color='red')
plt.title('Revenue Tuition vs. SAT scores')
plt.xlabel('SAT score')
plt.ylabel('Revenue tuition')
plt.show()

# SAT Scores and College Admission Rate

# Faculty Salary and Revenue Tuition

In [ ]:
average_rev_tuit_by_state = df.groupby('state')['revenue_tuit'].mean()
df_sorted_rev = df.sort_values(by='revenue_tuit', ascending=False)
top_10 = df_sorted_rev.head(10)

In [ ]:
average_salary_fac = df.groupby('state')
df_sorted_fac = average_rev_tuit_by_state.sort_values(by='avg_faculty_salary', ascending=False)
top_10 = df_sorted_fac.head(10)

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(average_salary_fac['state'], bins=2)
plt.title('Average Faculty by State')
plt.xlabel('State')
plt.ylabel('Average Faculty Salary')
plt.show()

Relationship between average faculty salary and revenue tuition, which will show the relationship and contrast between how much students pay and how much faculties get paid. 

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df['avg_faculty_salary'], df['revenue_tuit'], color='red')
plt.title('Revenue Tuition vs. Average Faculty Salary')
plt.xlabel('Average Faculty Salary')
plt.ylabel('Revenue tuition')
plt.show()

We seperate between public and private schools to see the difference between them. 

In [ ]:
private_public = df.groupby('control')['avg_faculty_salary'].mean()

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(private_public['control'], bin=2)
plt.title('Faculty Salary Difference between Public and Private Schools')
plt.xlabel(private_public['control'])
plt.ylabel('Average Faculty Salary')
plt.show()